In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:
"""
BRich Forex Trader - Prediction & Simulation Script - Final Project Group 13
"""

# =============================================================================
# Step 1 Imports
# =============================================================================
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import joblib
import yfinance as yf
import pandas_ta as ta
import math
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import warnings
import argparse

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#%%
# =============================================================================
# Step 2 Configuration Management Parameters
# =============================================================================
CFG = {
    # --- Data related settings ---
    "symbols_config": {
        "EURUSD=X": {"inverse": True, "name": "USDEUR=X"}, 
        "GBPUSD=X": {"inverse": True, "name": "USDGBP=X"}, 
        "USDJPY=X": {"inverse": False, "name": "USDJPY=X"}, 
    },
    "market_symbol": "SPY",         # Market benchmark (S&P 500)
    "start": "2014-01-01",          # Start date
    "end": "2025-06-15",            # End date
    "interval": "1d",               # Daily data
    "seq_len": 60,                  # Length of input sequence for the model (looking at past 60 days of data)
    "lr": 3e-4,                     # Learning Rate
    "device": "cuda" if torch.cuda.is_available() else "cpu", 

    # --- File path settings ---
    #"output_dir": os.path.join(os.path.dirname(os.path.abspath(__file__)), "Training_outputs"), # Path to the output directory generated by the training script
    "output_dir": os.path.join(os.getcwd(), "Training_outputs"),# Path to the output directory generated by the training script
    "model_filename": "best-model.ckpt",    # Name of the trained model file

    "simulation_days": 90,                  # Number of days for backtesting simulation
    "price_change_threshold_pct": 0.0005,   # Threshold percentage for triggering trades based on predicted price change
    "initial_capital": 10000,               # Initial capital for each currency pair
    "leverage": 20,                         # Leverage multiplier
    "position_unit_size": 1000,             # Base unit size for each trade
    "take_profit_ratio": 0.01,              # Take profit ratio
    "stop_loss_ratio": 0.005,               # Stop loss ratio
    "liquidation_threshold": 0.30,          # Equity threshold ratio for forced liquidation
    "max_position_units": 3,                # Maximum position units that can be added
    }

#%%
# =============================================================================
# Step 3 Data Processing Functions 
# =============================================================================
def download_and_process(sym: str, config: dict) -> pd.DataFrame:
    df = yf.download(sym, start=CFG["start"], end=CFG["end"], interval=CFG["interval"], progress=False, auto_adjust=False).reset_index()
    if df.empty: return pd.DataFrame()
    df.columns = [str(c[0] if isinstance(c, tuple) else c).lower() for c in df.columns]
    required_cols = {'date', 'open', 'high', 'low', 'close'}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"Data {sym} is missing required columns: {required_cols - set(df.columns)}")
    if 'volume' not in df.columns:
        df['volume'] = 0
    df = df[list(required_cols | {'volume'})].copy()
    df.dropna(inplace=True); df = df[df["close"] > 0]
    if config.get("inverse", False):
        df[['open', 'high', 'low', 'close']] = 1.0 / df[['open', 'high', 'low', 'close']]
        df.rename(columns={'high': 'temp_low', 'low': 'high'}, inplace=True)
        df.rename(columns={'temp_low': 'low'}, inplace=True)
    df["symbol"] = config["name"]; df['date'] = pd.to_datetime(df['date'])
    return df

def add_indicators(df: pd.DataFrame, market_df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.set_index('date', inplace=True)
    df.ta.ema(length=5, append=True); df.ta.ema(length=20, append=True)
    df.ta.rsi(length=14, append=True); df.ta.atr(length=14, append=True)
    df.ta.bbands(length=20, append=True); df.ta.stochrsi(length=14, append=True)
    df.ta.roc(length=10, append=True); df.ta.macd(append=True)
    df['volatility'] = df['close'].rolling(window=20).std()
    df.ta.ichimoku(append=True) # Add Ichimoku indicator
    df.reset_index(inplace=True)
    df['dayofweek'] = df['date'].dt.dayofweek; df['month'] = df['date'].dt.month
    merged = pd.merge(df, market_df[['date', 'close']], on='date', how='left', suffixes=('', '_market'))
    merged['market_corr'] = merged['close'].rolling(window=20).corr(merged['close_market'])
    df['market_corr'] = merged['market_corr']
    df['market_corr'] = df['market_corr'].fillna(0) # Safely fill NaN
    df.bfill(inplace=True); df.ffill(inplace=True); df.dropna(inplace=True)
    return df.reset_index(drop=True)

#%%
# =============================================================================
# Step 4 PyTorch Model Definition 
# =============================================================================
class MSConv(nn.Module):
    def __init__(self, i, o=32, k=(3,5,7)): super().__init__(); self.convs=nn.ModuleList([nn.Conv1d(i,o,j,padding=j//2)for j in k]); self.bn=nn.BatchNorm1d(o*len(k))
    def forward(self, x): return F.gelu(self.bn(torch.cat([c(x) for c in self.convs], dim=1)))
class PosEnc(nn.Module):
    def __init__(self, d, max_len=512): super().__init__(); pe=torch.zeros(max_len,d); p=torch.arange(0,max_len)[:,None]; div=torch.exp(torch.arange(0,d,2)*(-math.log(10000.0)/d)); pe[:,0::2]=torch.sin(p*div); pe[:,1::2]=torch.cos(p*div); self.register_buffer("pe",pe)
    def forward(self, x): return x + self.pe[:x.size(1)]
class MSConvTrans(nn.Module):
    def __init__(self, nf, T, d=128,h=4,nl=3): super().__init__(); self.ms=MSConv(nf);self.proj=nn.Conv1d(96,d,1);self.pos=PosEnc(d,T);el=nn.TransformerEncoderLayer(d,h,d*4,batch_first=True,norm_first=True,dropout=0.15);self.enc=nn.TransformerEncoder(el,nl)
    def forward(self,x): return self.enc(self.pos(self.proj(self.ms(x.permute(0,2,1))).permute(0,2,1)))
class HybridBackbone(nn.Module):
    def __init__(self, nf, T, d_t=128, d_g=128, n_g=2):
        super().__init__()
        self.transformer=MSConvTrans(nf,T,d=d_t)
        self.gru=nn.GRU(d_t,d_g,n_g,batch_first=True,bidirectional=True,dropout=0.25 if n_g>1 else 0)
        self.out_dim=d_g*2
    def forward(self, x):
        gru_out, _ = self.gru(self.transformer(x))
        return gru_out
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim):
        super().__init__()
        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = torch.sum(attention_weights * features, dim=1)
        return context_vector
class SupTask(pl.LightningModule):
    def __init__(self, n_features, seq_len, n_symbols, lr):
        super().__init__()
        self.save_hyperparameters()
        self.backbone=HybridBackbone(n_features,seq_len)
        self.sym_emb=nn.Embedding(n_symbols,16)
        self.attention_head = AttentionHead(self.backbone.out_dim, 128)
        flat_dim=self.backbone.out_dim + 16
        self.head=nn.Sequential(nn.Linear(flat_dim,512),nn.LayerNorm(512),nn.GELU(),nn.Dropout(0.6),nn.Linear(512,256),nn.LayerNorm(256),nn.GELU(),nn.Dropout(0.6))
        self.reg_head=nn.Linear(256,1)
    def forward(self, x, s):
        seq_features = self.backbone(x)
        context_vector = self.attention_head(seq_features)
        h=torch.cat([context_vector, self.sym_emb(s)], 1)
        return self.reg_head(self.head(h))

#%%
# =============================================================================
# Step 5 Trading Simulator Class 
# =============================================================================
class TradingSimulator:
    def __init__(self, external_prediction_data=None, external_real_data=None):
        """
        Initialize the simulator.
        1. Set file paths and computing device.
        2. Determine whether to use local model or external data (TA input).
        3. Load necessary auxiliary files.
        4. Prepare historical data for simulation.
        5. Initialize account status (capital, positions, etc.).
        """
        print("--- Initializing Trading Simulator ---")
        self.output_dir = os.path.abspath(CFG["output_dir"])
        self.model_path = os.path.join(self.output_dir, CFG["model_filename"])
        os.makedirs(self.output_dir, exist_ok=True)
        self.device = CFG["device"]

        self.external_prediction_data = external_prediction_data
        self.external_real_data = external_real_data

        if self.external_prediction_data is None:
            self._load_artifacts()
            self._prepare_simulation_data()
        else:
            self._prepare_external_data()

        self.sim_symbols = [conf['name'] for conf in CFG["symbols_config"].values()]
        self.n_symbols = len(self.sim_symbols)

        cleaned = [sym.replace("=X","") for sym in self.sim_symbols]
        self.currency = cleaned[0] if len(cleaned)==1 else "_".join(cleaned)

        self.capital = np.array([float(CFG["initial_capital"])] * self.n_symbols)
        self.position_size = np.zeros(self.n_symbols)
        self.entry_price = np.zeros(self.n_symbols)
        self.take_profit_price = np.zeros(self.n_symbols)
        self.stop_loss_price = np.zeros(self.n_symbols)
        self.history = []
        self.prediction_records = []
        print("✅ Initialization complete!")

    def _load_artifacts(self):
        """
        Load trained model, scaler, feature list, and other auxiliary files from disk.
        """
        try:
            self.feature_scalers = joblib.load(os.path.join(self.output_dir, 'feature_scalers.pkl'))
            self.feature_cols = joblib.load(os.path.join(self.output_dir, 'feature_cols.pkl'))
            self.symbols_trained = joblib.load(os.path.join(self.output_dir, 'symbols.pkl'))
            self.symbol_to_idx = {name: i for i, name in enumerate(self.symbols_trained)}
        except FileNotFoundError as e:
            raise FileNotFoundError(f"Necessary auxiliary files not found in '{self.output_dir}'. Please run the training script first. Error: {e}")

        n_features = len(self.feature_cols)
        n_symbols = len(self.symbols_trained)

        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model file not found: {self.model_path}")

        self.model = SupTask.load_from_checkpoint(
            self.model_path, map_location=self.device, n_features=n_features,
            seq_len=CFG['seq_len'], n_symbols=n_symbols, lr=CFG['lr']
        )
        self.model.to(self.device)
        self.model.eval()
        print(f"✅ Model and auxiliary files loaded from '{self.output_dir}'.")

    def _prepare_simulation_data(self):
        """
        Download and prepare data for local backtesting.
        """
        print("Downloading and preparing backtesting data ...")
        market_config = {"inverse": False, "name": CFG["market_symbol"]}
        market_df = download_and_process(CFG["market_symbol"], market_config)
        if market_df.empty:
            raise ValueError("Unable to download market data (SPY), cannot calculate market_corr feature.")

        all_dfs = []
        for sym, conf in CFG["symbols_config"].items():
            df = download_and_process(sym, conf)
            if not df.empty:
                df_with_indicators = add_indicators(df, market_df)
                all_dfs.append(df_with_indicators)

        if not all_dfs:
            raise ValueError("Unable to download or process data for all target currency pairs.")

        self.full_history_df = pd.concat(all_dfs).sort_values(["symbol", "date"]).reset_index(drop=True)
        unique_dates = sorted(self.full_history_df['date'].unique())
        
        if len(unique_dates) <= CFG["simulation_days"]:
             raise ValueError(f"Number of data days ({len(unique_dates)}) is insufficient for {CFG['simulation_days']} days of backtesting.")

        self.simulation_dates = unique_dates[-CFG["simulation_days"]-1:]
        self.simulation_df = self.full_history_df[self.full_history_df['date'].isin(self.simulation_dates)].copy()
        print(f"Backtesting will be conducted from {self.simulation_dates[0].date()} to {self.simulation_dates[-2].date()}.")

    def _prepare_external_data(self):
        """
        Function: Process data from external Excel files (TA grading mode).
        """
        print("Processing data provided by the TA...")
        self.sim_symbols = [conf['name'] for conf in CFG["symbols_config"].values()]
        num_days = self.external_real_data.shape[0]
        dates = pd.to_datetime(pd.date_range(end=pd.Timestamp.now().date(), periods=num_days, freq='B'))

        data_frames = []
        for i, symbol in enumerate(self.sim_symbols):
            df = pd.DataFrame({
                'date': dates, 'symbol': symbol,
                'open': self.external_real_data[:, i], 'high': self.external_real_data[:, i],
                'low': self.external_real_data[:, i], 'close': self.external_real_data[:, i]
            })
            data_frames.append(df)
        self.simulation_df = pd.concat(data_frames, ignore_index=True)
        self.simulation_dates = sorted(self.simulation_df['date'].unique())
        print(f"Simulation using external data for a total of {len(self.simulation_dates)-1} trading days.")

    def _get_input_sequence(self, symbol: str, current_date: pd.Timestamp):
        """
        Prepare the standardized input sequence required for model prediction for the specified product and date.
        """
        df_before_current = self.full_history_df[(self.full_history_df['symbol'] == symbol) & (self.full_history_df['date'] < current_date)]
        if len(df_before_current) < CFG["seq_len"]: return None, None
        
        seq_df = df_before_current.tail(CFG["seq_len"]).copy()
        current_price = seq_df.iloc[-1]['close'] 

        valid_feature_cols = [col for col in self.feature_cols if col in seq_df.columns]
        if symbol in self.feature_scalers:
            seq_df[valid_feature_cols] = self.feature_scalers[symbol].transform(seq_df[valid_feature_cols])
        
        input_tensor = torch.tensor(seq_df[valid_feature_cols].values, dtype=torch.float32).unsqueeze(0).to(self.device)
        return input_tensor, current_price

    def _predict_price(self, symbol: str, current_date: pd.Timestamp):
        """
        Execute a single price prediction, converting the model's output log return to absolute price.
        """
        input_seq, current_price = self._get_input_sequence(symbol, current_date)
        if input_seq is None: return None

        symbol_idx = torch.tensor([self.symbol_to_idx[symbol]], dtype=torch.long).to(self.device)
        with torch.no_grad():
            predicted_log_return = self.model(input_seq, symbol_idx).item()

        predicted_price = current_price * math.exp(predicted_log_return)
        return predicted_price

    def run(self):
        """
        Execute the main backtesting loop, including your custom trading logic.
        """
        print("\n--- 🚀 Starting Trading Simulation (including position adding logic) ---")
        for day_idx, date in enumerate(self.simulation_dates[:-1]):
            print(f"\n====================== Trading Day: {date.date()} ======================")
            daily_history = {'date': date, 'symbols': {}}
            daily_capital = self.capital.copy()
            total_realized_pnl_day = 0
            for i, sym in enumerate(self.sim_symbols):
                print(f"\n--- Symbol: {sym.replace('=X', ''): <7} ---")
                current_price = None
                if self.external_real_data is None:
                    current_price_row = self.simulation_df[(self.simulation_df['date'] == date) & (self.simulation_df['symbol'] == sym)]
                    if current_price_row.empty:
                        print(f"Status: No data for today")
                        continue
                    current_price = current_price_row.iloc[0]['close']
                else:
                    if day_idx < self.external_real_data.shape[0]:
                        current_price = self.external_real_data[day_idx, i]
                    else:
                        print(f"Status: No external data for today")
                        continue
                daily_history['symbols'][sym] = {
                    'actual_price': current_price, 'predicted_price': None, 'floating_pnl': 0.0,
                    'position_size': self.position_size[i], 'action': 'HOLD'
                }
                if self.position_size[i] != 0:
                    floating_pnl = self.position_size[i] * (current_price - self.entry_price[i])
                    daily_history['symbols'][sym]['floating_pnl'] = floating_pnl
                    equity_for_check = daily_capital[i] + floating_pnl
                    margin_required = (abs(self.position_size[i]) * self.entry_price[i]) / CFG["leverage"]
                    exit_reason = None
                    maintenance_margin = margin_required * CFG["liquidation_threshold"]
                    if equity_for_check < maintenance_margin: exit_reason = f"LIQUIDATION (Equity ${equity_for_check:.2f} < Maintenance Margin ${maintenance_margin:.2f})"
                    elif self.position_size[i] > 0 and current_price >= self.take_profit_price[i]: exit_reason = "TAKE_PROFIT"
                    elif self.position_size[i] > 0 and current_price <= self.stop_loss_price[i]: exit_reason = "STOP_LOSS"
                    elif self.position_size[i] < 0 and current_price <= self.take_profit_price[i]: exit_reason = "TAKE_PROFIT"
                    elif self.position_size[i] < 0 and current_price >= self.stop_loss_price[i]: exit_reason = "STOP_LOSS"
                    if exit_reason:
                        realized_pnl_on_close = floating_pnl
                        daily_capital[i] += realized_pnl_on_close
                        total_realized_pnl_day += realized_pnl_on_close
                        print(f"🔴 Closing: {exit_reason} | Price: {current_price:.4f} | Realized PnL: ${realized_pnl_on_close:.2f}")
                        self.position_size[i] = 0.0
                        self.entry_price[i], self.take_profit_price[i], self.stop_loss_price[i] = 0, 0, 0
                        daily_history['symbols'][sym]['action'] = 'CLOSE'
                        daily_history['symbols'][sym]['realized_pnl'] = realized_pnl_on_close
                        daily_history['symbols'][sym]['floating_pnl'] = 0.0
                predicted_price = None
                if self.external_prediction_data is None and sym in self.symbol_to_idx:
                    predicted_price = self._predict_price(sym, date)
                elif self.external_prediction_data is not None:
                     if day_idx < self.external_prediction_data.shape[0]:
                         predicted_price = self.external_prediction_data[day_idx, i]
                daily_history['symbols'][sym]['predicted_price'] = predicted_price
                if predicted_price is not None:
                    print(f"Price: Current {current_price:.4f}, Model Prediction {predicted_price:.4f}")
                    next_day_date = self.simulation_dates[day_idx + 1]
                    actual_next_price = None
                    if self.external_real_data is None:
                        next_day_price_row = self.simulation_df[(self.simulation_df['date'] == next_day_date) & (self.simulation_df['symbol'] == sym)]
                        if not next_day_price_row.empty: actual_next_price = next_day_price_row.iloc[0]['close']
                    else:
                        if day_idx + 1 < len(self.external_real_data): actual_next_price = self.external_real_data[day_idx + 1, i]
                    if actual_next_price is not None:
                        self.prediction_records.append({"symbol": sym, "predicted": predicted_price, "actual": actual_next_price})
                    predicted_change_pct = (predicted_price - current_price) / current_price
                    current_units = abs(self.position_size[i]) / CFG["position_unit_size"] if self.position_size[i] != 0 else 0
                    if self.position_size[i] == 0:
                        if predicted_change_pct > CFG["price_change_threshold_pct"]:
                            self.position_size[i] = CFG["position_unit_size"]
                            self.entry_price[i] = current_price
                            self.take_profit_price[i] = current_price * (1 + CFG["take_profit_ratio"])
                            self.stop_loss_price[i] = current_price * (1 - CFG["stop_loss_ratio"])
                            daily_history['symbols'][sym]['action'] = 'OPEN_LONG'
                            print(f"🟢 Opening: LONG | Price: {current_price:.4f} | TP: {self.take_profit_price[i]:.4f} | SL: {self.stop_loss_price[i]:.4f}")
                        elif predicted_change_pct < -CFG["price_change_threshold_pct"]:
                            self.position_size[i] = -CFG["position_unit_size"]
                            self.entry_price[i] = current_price
                            self.take_profit_price[i] = current_price * (1 - CFG["take_profit_ratio"])
                            self.stop_loss_price[i] = current_price * (1 + CFG["stop_loss_ratio"])
                            daily_history['symbols'][sym]['action'] = 'OPEN_SHORT'
                            print(f"🟢 Opening: SHORT | Price: {current_price:.4f} | TP: {self.take_profit_price[i]:.4f} | SL: {self.stop_loss_price[i]:.4f}")
                    else:
                        is_long = self.position_size[i] > 0
                        if (is_long and predicted_change_pct > CFG["price_change_threshold_pct"] and current_units < CFG["max_position_units"]):
                            new_cost = CFG["position_unit_size"] * current_price
                            total_current_value = self.position_size[i] * self.entry_price[i]
                            self.position_size[i] += CFG["position_unit_size"]
                            self.entry_price[i] = (total_current_value + new_cost) / self.position_size[i]
                            daily_history['symbols'][sym]['action'] = 'ADD_LONG'
                            print(f"🔼 Adding: LONG | New Average Price: {self.entry_price[i]:.4f} | Current Units: {current_units + 1}")
                        elif (not is_long and predicted_change_pct < -CFG["price_change_threshold_pct"] and current_units < CFG["max_position_units"]):
                            new_cost = CFG["position_unit_size"] * current_price
                            total_current_value = abs(self.position_size[i]) * self.entry_price[i]
                            self.position_size[i] -= CFG["position_unit_size"]
                            self.entry_price[i] = (total_current_value + new_cost) / abs(self.position_size[i])
                            daily_history['symbols'][sym]['action'] = 'ADD_SHORT'
                            print(f"🔽 Adding: SHORT | New Average Price: {self.entry_price[i]:.4f} | Current Units: {current_units + 1}")
                        elif (is_long and predicted_change_pct < 0) or (not is_long and predicted_change_pct > 0):
                            final_pnl = self.position_size[i] * (current_price - self.entry_price[i])
                            daily_capital[i] += final_pnl
                            total_realized_pnl_day += final_pnl
                            print(f"🔴 Closing: Signal Reversal | Price: {current_price:.4f} | Realized PnL: ${final_pnl:.2f}")
                            self.position_size[i] = 0.0
                            self.entry_price[i], self.take_profit_price[i], self.stop_loss_price[i] = 0, 0, 0
                            daily_history['symbols'][sym]['action'] = 'CLOSE'
                            daily_history['symbols'][sym]['realized_pnl'] = final_pnl
                            daily_history['symbols'][sym]['floating_pnl'] = 0.0
                else:
                    print(f"Price: Current {current_price:.4f}, Unable to predict")
                if self.position_size[i] > 0: print(f"Status: Holding LONG position | Size: {self.position_size[i]} | Average Price: {self.entry_price[i]:.4f}")
                elif self.position_size[i] < 0: print(f"Status: Holding SHORT position | Size: {self.position_size[i]} | Average Price: {self.entry_price[i]:.4f}")
                else:
                    if daily_history['symbols'][sym]['action'] == 'HOLD': print(f"Status: No position")
            final_floating_pnl_day = 0
            for i, sym in enumerate(self.sim_symbols):
                if self.position_size[i] != 0:
                    current_price = None
                    if self.external_real_data is None:
                       current_price_row = self.simulation_df[(self.simulation_df['date'] == date) & (self.simulation_df['symbol'] == sym)]
                       if not current_price_row.empty: current_price = current_price_row.iloc[0]['close']
                    else:
                        if day_idx < self.external_real_data.shape[0]: current_price = self.external_real_data[day_idx, i]
                    if current_price is not None: final_floating_pnl_day += self.position_size[i] * (current_price - self.entry_price[i])
            daily_history['total_equity'] = np.sum(self.capital) + total_realized_pnl_day + final_floating_pnl_day
            self.history.append(daily_history)
            self.capital = daily_capital.copy()
        print("\n--- Simulation ended, settling all remaining positions... ---")
        last_trading_day_date = self.simulation_dates[-2]
        for i, sym in enumerate(self.sim_symbols):
            if self.position_size[i] != 0:
                last_price = None
                if self.external_real_data is None:
                    last_day_prices_df = self.simulation_df[(self.simulation_df['date'] == last_trading_day_date) & (self.simulation_df['symbol'] == sym)]
                    if not last_day_prices_df.empty: last_price = last_day_prices_df.iloc[0]['close']
                else:
                    if CFG["simulation_days"]-1 < self.external_real_data.shape[0]: last_price = self.external_real_data[CFG["simulation_days"] - 1, i]
                if last_price is not None:
                    final_pnl = self.position_size[i] * (last_price - self.entry_price[i])
                    self.capital[i] += final_pnl
                    print(f"Settling {sym} position, Realized PnL: ${final_pnl:.2f} (at price: {last_price:.4f})")
        initial_total_capital = float(CFG["initial_capital"]) * self.n_symbols
        final_total_capital = np.sum(self.capital)
        roi = (final_total_capital / initial_total_capital - 1) * 100 if initial_total_capital > 0 else 0
        print("\n--- 📊 Final Simulation Results ---")
        for i, sym in enumerate(self.sim_symbols): print(f"Currency Pair {sym: <9}: Final Capital ${self.capital[i]:.2f}")
        print("-" * 30)
        print(f"Initial Total Capital: ${initial_total_capital:.2f}")
        print(f"Final Total Capital: ${final_total_capital:.2f}")
        print(f"Total Return on Investment (ROI): {roi:.2f}%")
        self._calculate_and_print_accuracy()
        self.export_results()
        # self.plot_results()

    def _calculate_and_print_accuracy(self):
        """
        Calculate the model's prediction accuracy metrics (MAE, MAPE).
        """
        if not self.prediction_records:
            print("\nNo prediction records available for accuracy analysis.")
            return
        records_df = pd.DataFrame(self.prediction_records)
        print("\n--- 🎯 Model Prediction Accuracy Analysis ---")
        mae = np.mean(np.abs(records_df['predicted'] - records_df['actual']))
        mape = np.mean(np.abs((records_df['predicted'] - records_df['actual']) / records_df['actual'])) * 100
        print(f"Overall Accuracy:\n - Mean Absolute Error (MAE): {mae:.5f}\n - Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
        for sym in self.sim_symbols:
            sym_df = records_df[records_df['symbol'] == sym]
            if not sym_df.empty:
                mae_sym = np.mean(np.abs(sym_df['predicted'] - sym_df['actual']))
                mape_sym = np.mean(np.abs((sym_df['predicted'] - sym_df['actual']) / sym_df['actual'])) * 100
                print(f"\nCurrency Pair {sym.replace('=X', ''): <7}:\n - MAE: {mae_sym:.5f}\n - MAPE: {mape_sym:.2f}%")
        # Save simulation accuracy to its own CSV (won't overwrite training file)
        sim_acc_csv = os.path.join(CFG["output_dir"], "simulation_prediction_accuracy.csv")
        new_acc = records_df.copy()
        if os.path.exists(sim_acc_csv):
            old_acc = pd.read_csv(sim_acc_csv)
            combined = pd.concat([old_acc, new_acc], ignore_index=True).drop_duplicates(ignore_index=True)
        else:
            combined = new_acc
        combined.to_csv(sim_acc_csv, index=False, encoding="utf-8-sig")
        print(f"✅ Simulation accuracy saved to: {sim_acc_csv}")

    def plot_results(self):
        """
        Visualize simulation results, plotting trading analysis charts and equity curve charts.
        """
        # [Modify] Change all chart elements to English
        history_df = self.get_prediction_series()
        if history_df.empty:
            print("No historical data available for plotting.")
            return
        
        n_symbols = len(self.sim_symbols)
        # Plot the first chart: Trading analysis chart
        fig1, axes = plt.subplots(n_symbols, 1, figsize=(15, 6 * n_symbols), sharex=True, dpi=100)
        if n_symbols == 1: axes = [axes]
        
        for i, sym in enumerate(self.sim_symbols):
            ax = axes[i]
            sym_history = history_df[history_df['symbol'] == sym].reset_index(drop=True)
            ax.plot(sym_history.index, sym_history['actual_price'], label='Actual Price', color='black', lw=1.5)
            ax.plot(sym_history.index, sym_history['predicted_price'], label='Predicted Price', color='dodgerblue', linestyle='--', alpha=0.8)
            
            actions = {
                'OPEN_LONG': ('^', 'green', 120, 5, 'Open Long'), 
                'OPEN_SHORT': ('v', 'red', 120, 5, 'Open Short'),
                'ADD_LONG': ('P', 'lime', 100, 4, 'Add Long'), 
                'ADD_SHORT': ('X', 'salmon', 100, 4, 'Add Short'),
                'CLOSE': ('D', 'blue', 80, 5, 'Close Position')}
            
            for action_name, (marker, color, size, zorder, label) in actions.items():
                action_df = sym_history[sym_history['action'] == action_name]
                if not action_df.empty:
                    ax.scatter(action_df.index, action_df['actual_price'], label=label, marker=marker, color=color, s=size, edgecolors='k', zorder=zorder)
            
            ax.set_title(f'Prediction and Trade Analysis for {sym.replace("=X", "")}')
            ax.set_ylabel('Price')
            ax.legend()
            ax.grid(True, linestyle='--', alpha=0.6)
        
        axes[-1].set_xlabel('Simulation Days')
        fig1.suptitle('Trading Simulation Analysis', fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        fig1.savefig(os.path.join(self.output_dir, f"trading_simulation_analysis_{self.currency}.png"))
        print(f"✅ Trading analysis chart saved to: {os.path.join(self.output_dir, f'trading_simulation_analysis_{self.currency}.png')}")
        plt.show()

        # Plot the second chart: Equity curve chart
        fig2, ax2 = plt.subplots(figsize=(15, 7), dpi=100)
        if self.history:
            total_equity_curve = [h['total_equity'] for h in self.history if 'total_equity' in h]
            initial_total_capital = float(CFG["initial_capital"]) * self.n_symbols
            ax2.plot(total_equity_curve, label='Total Equity (Capital + Floating PnL)', color='royalblue', lw=2)
            ax2.axhline(y=initial_total_capital, label='Initial Total Capital', color='black', linestyle=':')
        
        ax2.set_title('Trading Simulation Equity Curve')
        ax2.set_xlabel('Simulation Days')
        ax2.set_ylabel('Equity (USD)')
        ax2.legend()
        ax2.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()
        fig2.savefig(os.path.join(self.output_dir, f"equity_curve_{self.currency}.png"))
        print(f"✅ Equity curve chart saved to: {os.path.join(self.output_dir, f'equity_curve_{self.currency}.png')}")
        plt.show()

    def get_prediction_series(self) -> pd.DataFrame:
        """
        Convert stored historical records (by day) into a flattened DataFrame for easy plotting and exporting.
        """
        if not self.history:
            print("Warning: No historical records available for extraction. Please run run() first.")
            return pd.DataFrame()
        flat_history = []
        for daily_record in self.history:
            date = daily_record['date']
            for symbol, details in daily_record['symbols'].items():
                flat_history.append({
                    'date': date, 'symbol': symbol, 'actual_price': details.get('actual_price'),
                    'predicted_price': details.get('predicted_price'), 'floating_pnl': details.get('floating_pnl', 0.0),
                    'realized_pnl': details.get('realized_pnl', 0.0), 'position_size': details.get('position_size', 0.0),
                    'action': details.get('action', 'HOLD')})
        return pd.DataFrame(flat_history)

    def export_results(self):
        """
        Export simulation results to various file formats for GUI use or backup.
        """
        print("\n--- 💾 Exporting simulation results ---")
        history_df = self.get_prediction_series()
        if not history_df.empty:
            gui_json_path = os.path.join(self.output_dir, "simulation_details.json")
            history_df_json = history_df.copy()
            history_df_json['date'] = history_df_json['date'].dt.strftime('%Y-%m-%d')
            # — append to existing JSON if present, then drop duplicates
            if os.path.exists(gui_json_path):
                with open(gui_json_path, 'r', encoding='utf-8') as f:
                    old_records = json.load(f)
                old_df = pd.DataFrame(old_records)
                combined_json_df = pd.concat([old_df, history_df_json], ignore_index=True)
            else:
                combined_json_df = history_df_json
            # remove duplicate entries per date+symbol, keeping the latest run
            combined_json_df = combined_json_df.drop_duplicates(
                subset=['date', 'symbol'], keep='last', ignore_index=True
            )
            with open(gui_json_path, 'w', encoding='utf-8') as f:
                json.dump(combined_json_df.to_dict('records'), f, ensure_ascii=False, indent=4)
            print(f"✅ GUI trading details appended to: {gui_json_path}")

            # — append new rows to CSV backup, then drop duplicates
            csv_path = os.path.join(self.output_dir, "simulation_details.csv")
            if os.path.exists(csv_path):
                old_csv = pd.read_csv(csv_path)
                combined_csv = pd.concat([old_csv, history_df], ignore_index=True)
            else:
                combined_csv = history_df
            # remove duplicate entries per date+symbol, keeping the latest run
            combined_csv = combined_csv.drop_duplicates(
                subset=['date', 'symbol'], keep='last', ignore_index=True
            )
            combined_csv.to_csv(csv_path, index=False, encoding='utf-8-sig')
            print(f"✅ [Backup] Daily trading details appended to: {csv_path}")
        if self.history:
            equity_data = [{'date': r['date'].strftime('%Y-%m-%d'), 'total_equity': r['total_equity']} for r in self.history if 'total_equity' in r]
            equity_df = pd.DataFrame(equity_data)
            equity_json_path = os.path.join(self.output_dir, "equity_curve.json")
            with open(equity_json_path, 'w', encoding='utf-8') as f:
                json.dump(equity_df.to_dict('records'), f, ensure_ascii=False, indent=4)
            print(f"✅ GUI equity curve data saved to: {equity_json_path}")
        if self.prediction_records:
            new_acc = pd.DataFrame(self.prediction_records)
            sim_acc_csv = os.path.join(self.output_dir, "simulation_prediction_accuracy.csv")
            if os.path.exists(sim_acc_csv):
                old_acc = pd.read_csv(sim_acc_csv)
                combined = pd.concat([old_acc, new_acc], ignore_index=True).drop_duplicates(ignore_index=True)
            else:
                combined = new_acc
            combined.to_csv(sim_acc_csv, index=False, encoding="utf-8-sig")
            print(f"✅ [Backup] Simulation prediction accuracy appended to: {sim_acc_csv}")

#%%
# =============================================================================
# Step 6 Main Execution
# =============================================================================
if __name__ == "__main__":
    """
    Main entry point of the program:
    1. First check if there are external data files provided by the TA, if so, run in TA mode.
    2. If there are no external files, check if there is a locally trained model.
    3. If neither exists, prompt the user to run training first.
    """
    PRED_FX_FILE = 'fake_fx_data.xlsx'   # TA input
    REAL_FX_FILE = 'fx_data.xlsx'

    if os.path.exists(PRED_FX_FILE) and os.path.exists(REAL_FX_FILE):
        print(f"Detected external data files, running in TA mode...")
        try:
            df_pre = pd.read_excel(PRED_FX_FILE, header=None)
            ta_pre_fx_data = df_pre.values
            df_real = pd.read_excel(REAL_FX_FILE, header=None)
            ta_real_fx_data = df_real.values
            if ta_pre_fx_data.shape != ta_real_fx_data.shape:
                print(f"Warning: The shapes of predicted data and actual data do not match! " \
                      f"Pred: {ta_pre_fx_data.shape}, Real: {ta_real_fx_data.shape}. " \
                       "Please check the Excel files.")
            print("Running simulation using data provided by the TA.")
            simulator = TradingSimulator(external_prediction_data=ta_pre_fx_data, external_real_data=ta_real_fx_data)
            simulator.run()
        except Exception as e:
            print(f"Error reading or processing TA Excel files: {e}")

    elif os.path.exists(os.path.join(CFG["output_dir"], CFG["model_filename"])):
        print("No TA input files found, running simulation using local model and Yahoo Finance data.")
        try:
            parser = argparse.ArgumentParser()
            parser.add_argument("--simulation_days", type=int,   default=CFG["simulation_days"])
            parser.add_argument("--initial_capital", type=float, default=CFG["initial_capital"])
            parser.add_argument("--leverage", type=float,        default=CFG["leverage"])
            parser.add_argument("--currency", type=str,          default=None,
                                help="Single currency symbol to simulate (e.g. EURUSD=X)")
            args, unknown = parser.parse_known_args() 

            # if GUI passed a currency, override the hard-coded symbols_config
            if args.currency:
                CFG["symbols_config"] = {
                    args.currency: {"inverse": False, "name": args.currency}}
            CFG["simulation_days"]  = args.simulation_days
            CFG["initial_capital"]  = args.initial_capital
            CFG["leverage"]         = args.leverage

            simulator = TradingSimulator()
            simulator.run()
        except Exception as e:
            print(f"Error running simulation with local model: {e}")
    else:
        print(f"Error: Model file '{CFG['model_filename']}' not found in '{CFG['output_dir']}'.")
        print("Please successfully run the training script first, or place the TA's 'fake_fx_data.xlsx' and 'fx_data.xlsx' files in the same directory.")

# End of Code

No TA input files found, running simulation using local model and Yahoo Finance data.
--- Initializing Trading Simulator ---
✅ Model and auxiliary files loaded from 'C:\Users\USER\Desktop\Machine Learning\Group_13_BRich Forex trader\Training_outputs'.
Backtesting will be conducted from 2025-02-05 to 2025-06-12.
✅ Initialization complete!

--- 🚀 Starting Trading Simulation (including position adding logic) ---

====================== Trading Day: 2025-02-05 ======================

--- Symbol: USDEUR  ---
Price: Current 0.9641, Model Prediction 0.9678
🟢 Opening: LONG | Price: 0.9641 | TP: 0.9737 | SL: 0.9592
Status: Holding LONG position | Size: 1000.0 | Average Price: 0.9641

--- Symbol: USDGBP  ---
Price: Current 0.8014, Model Prediction 0.8040
🟢 Opening: LONG | Price: 0.8014 | TP: 0.8094 | SL: 0.7974
Status: Holding LONG position | Size: 1000.0 | Average Price: 0.8014

--- Symbol: USDJPY  ---
Price: Current 154.1330, Model Prediction 155.2010
🟢 Opening: LONG | Price: 154.1330 | TP: 15

In [ ]:
!jupyter nbconvert --to script Final_simulate.ipynb